# ABN

Les données historiques de la station Niamey

In [1]:
%load_ext jupyter_black

In [2]:
import os
from pathlib import Path

from dotenv import load_dotenv
import datetime
import xarray as xr
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from ochanticipy import (
    create_country_config,
    CodAB,
    GeoBoundingBox,
    GlofasForecast,
    GlofasReanalysis,
    GlofasReforecast,
)

pio.renderers.default = "notebook"

/Users/tdowning/.pyenv/versions/pa-aa-ner-flooding/lib/python3.11/site-packages/quantulum3/classifier.py:28: UserWarning: Classifier dependencies not installed. Run pip install quantulum3[classifier] to install them. The classifer helps to dissambiguate units.
  warnings.warn(


In [3]:
load_dotenv()

RAW_DIR = Path(os.environ["OAP_DATA_DIR"]) / "public/raw/ner"
ABN_PATH = RAW_DIR / "abn/Données_ Niamey2005_2022.xlsx"

adm1_sel = ["Tillabéri", "Niamey", "Dosso", "Maradi"]
startdate = datetime.date(2005, 1, 1)
enddate = datetime.date(2024, 1, 1)

country_config = create_country_config(iso3="ner")
codab = CodAB(country_config=country_config)
gdf_adm1 = codab.load(admin_level=1)
gdf_aoi = gdf_adm1[gdf_adm1["adm_01"].isin(adm1_sel)]
geobb = GeoBoundingBox.from_shape(gdf_aoi)

In [17]:
# read ABN data
# note - this is originally from Niger-HYCOS 

df_abn = pd.read_excel(ABN_PATH, skiprows=[0, 1], index_col=0)
df_abn

,Water Level (cm),Discharges (m3/s)
Date,,
2005-01-01,459.667,1367.244
2005-01-02,457.667,1354.711
2005-01-03,455.667,1342.178
2005-01-04,453.667,1329.644
2005-01-05,451.000,1312.933
...,...,...
2022-07-28,298.000,355.000
2022-07-29,298.000,355.000
2022-07-30,336.000,485.500


In [156]:
fig = px.line(df_abn, y="Water Level (cm)")
fig.update_layout(template="simple_white")
fig.show()

In [111]:
df_plot = df_abn.copy()
df_plot["year"] = df_abn.index.year
px.scatter(
    df_abn, x="Discharges (m3/s)", y="Water Level (cm)", color="year"
).show()

In [32]:
df_abn.corr(method="pearson")

,Water Level (cm),Discharges (m3/s)
Water Level (cm),1.000000,0.978188
Discharges (m3/s),0.978188,1.000000


In [43]:
# calculate return

df_max = df_abn.groupby(df_abn.index.year)["Water Level (cm)"].max()
total_years = len(df_max)

return_years = [1.5, 2, 3, 5, 10]

df_count_yearly = pd.DataFrame()
df_return_periods = pd.DataFrame(index=return_years)

dff = df_max
print(dff)
quants = np.linspace(0, 1, 101)
levels = dff.quantile(quants)
#     print(levels)
for level in levels:
    dfff = dff[dff >= level]
    years = "<br>".join(
        [str(x) for x in dfff.sort_index(ascending=False).index]
    )
    count = len(dfff)
    df_add = pd.DataFrame(
        {
            "level": level,
            "count": count,
            "years": years,
            "return": total_years / count,
        },
        index=[0],
    )
    df_count_yearly = pd.concat([df_count_yearly, df_add], ignore_index=True)
df_i = df_count_yearly
df_return_periods["level"] = np.interp(
    return_years, df_i["return"], df_i["level"]
)

df_count_yearly["return"] = len(df_max) / df_count_yearly["count"]

Date
2005    497.000
2006    512.000
2007    508.000
2008    525.000
2009    528.667
2010    565.000
2011    538.000
2012    616.667
2013    611.750
2014    512.000
2015    579.667
2016    602.667
2017    595.667
2018    581.000
2019    636.000
2020    699.333
2021    582.000
2022    554.000
Name: Water Level (cm), dtype: float64


In [45]:
df_count_yearly

,level,count,years,return
0,497.00000,18,2022<br>2021<br>2020<br>2019<br>2018<br>2017<b...,1.000000
1,498.87000,17,2022<br>2021<br>2020<br>2019<br>2018<br>2017<b...,1.058824
2,500.74000,17,2022<br>2021<br>2020<br>2019<br>2018<br>2017<b...,1.058824
3,502.61000,17,2022<br>2021<br>2020<br>2019<br>2018<br>2017<b...,1.058824
4,504.48000,17,2022<br>2021<br>2020<br>2019<br>2018<br>2017<b...,1.058824
...,...,...,...,...
96,656.26656,1,2020,18.000000
97,667.03317,1,2020,18.000000
98,677.79978,1,2020,18.000000
99,688.56639,1,2020,18.000000


In [155]:
# plot return period station

df_plot = df_count_yearly

min_year = df_abn.index.year.min()

x_max = 10
y_max = df_plot[df_plot["return"] <= x_max]["level"].max()
y_min = df_plot["level"].min()
fig = px.line(df_plot, x="return", y="level")

for year in return_years:
    level = df_return_periods.loc[year, "level"]
    fig.add_trace(
        go.Scatter(
            x=[1, x_max],
            y=[level, level],
            mode="lines+text",
            text=[None, f"{year}-year"],
            line=dict(width=1, dash="dash", color="black"),
            textposition="top left",
        )
    )

fig.update_xaxes(range=(1, x_max), title="Return period (years)")
fig.update_yaxes(
    range=(y_min, y_max), title="One-day flow rate (m<sup>3</sup>/s)"
)

fig.update_layout(
    template="simple_white",
    title=f"ABN période de retour (depuis {min_year})",
    showlegend=False,
)

In [118]:
# load GloFAS reanalysis

start_date = df_abn.index.min().date()
end_date = df_abn.index.max().date()

g_re = GlofasReanalysis(
    country_config=country_config,
    geo_bounding_box=geobb,
    start_date=start_date,
    end_date=end_date,
)
g_re.download()
g_re.process()
ds = g_re.load()
df_re = ds.to_dataframe()["Niamey"]

In [119]:
df_re

time
2005-01-01    2662.414062
2005-01-02    2625.687500
2005-01-03    2588.812500
2005-01-04    2551.812500
2005-01-05    2514.828125
                 ...     
2022-12-27    3960.984375
2022-12-28    3884.070312
2022-12-29    3808.429688
2022-12-30    3734.250000
2022-12-31    3661.539062
Name: Niamey, Length: 6574, dtype: float32

In [144]:
# GloFAS 4.0 reanalysis

g_re4 = GlofasReanalysis(
    country_config=country_config,
    geo_bounding_box=geobb,
    start_date=datetime.date(2020, 1, 1),
    end_date=datetime.date(2020, 12, 31),
)
g_re4.process()
da = g_re4.load()
df_re4 = da.to_dataframe()
df_re4

,Alcongui,Boss Bangou,Dargol A Kakassi,Dargol At Tera,Diongore Amont,Garbe-Kourou,Gorouol At Dolbel,Goulbi De Maradi A Nielloua,Kandadji,Majia At Tsernaoua,Niamey,W
time,,,,,,,,,,,,
2020-01-01,0.328125,0.031250,0.0,0.0,0.0,0.093750,0.0,0.000000,1162.750000,0.000000,1238.046875,0.0
2020-01-02,0.125000,0.031250,0.0,0.0,0.0,0.093750,0.0,0.000000,1162.984375,0.000000,1237.250000,0.0
2020-01-03,0.125000,0.031250,0.0,0.0,0.0,0.093750,0.0,0.000000,1131.921875,0.000000,1203.875000,0.0
2020-01-04,0.109375,0.015625,0.0,0.0,0.0,0.078125,0.0,0.000000,1102.437500,0.000000,1171.968750,0.0
2020-01-05,0.109375,0.015625,0.0,0.0,0.0,0.078125,0.0,0.000000,1074.453125,0.000000,1141.562500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,0.218750,0.015625,0.0,0.0,0.0,0.062500,0.0,0.015625,1291.078125,0.781250,1367.640625,0.0
2020-12-28,0.218750,0.015625,0.0,0.0,0.0,0.062500,0.0,0.015625,1258.875000,0.765625,1332.468750,0.0
2020-12-29,0.203125,0.015625,0.0,0.0,0.0,0.062500,0.0,0.015625,1228.265625,0.765625,1299.078125,0.0


In [145]:
df_compare = df_abn.merge(df_re, left_index=True, right_index=True)
df_compare = df_compare.merge(
    df_re4, left_index=True, right_index=True, suffixes=["3", "4"]
)
df_compare

,Water Level (cm),Discharges (m3/s),year,Niamey3,Alcongui,Boss Bangou,Dargol A Kakassi,Dargol At Tera,Diongore Amont,Garbe-Kourou,Gorouol At Dolbel,Goulbi De Maradi A Nielloua,Kandadji,Majia At Tsernaoua,Niamey4,W
2020-01-01,554.0,1595.0,2020,3795.250000,0.328125,0.031250,0.0,0.0,0.0,0.093750,0.0,0.000000,1162.750000,0.000000,1238.046875,0.0
2020-01-02,553.5,1591.5,2020,3734.937500,0.125000,0.031250,0.0,0.0,0.0,0.093750,0.0,0.000000,1162.984375,0.000000,1237.250000,0.0
2020-01-03,553.5,1591.5,2020,3675.000000,0.125000,0.031250,0.0,0.0,0.0,0.093750,0.0,0.000000,1131.921875,0.000000,1203.875000,0.0
2020-01-04,552.5,1585.0,2020,3615.453125,0.109375,0.015625,0.0,0.0,0.0,0.078125,0.0,0.000000,1102.437500,0.000000,1171.968750,0.0
2020-01-05,553.0,1588.0,2020,3556.265625,0.109375,0.015625,0.0,0.0,0.0,0.078125,0.0,0.000000,1074.453125,0.000000,1141.562500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,570.0,1699.0,2020,4472.960938,0.218750,0.015625,0.0,0.0,0.0,0.062500,0.0,0.015625,1291.078125,0.781250,1367.640625,0.0
2020-12-28,571.0,1706.0,2020,4390.789062,0.218750,0.015625,0.0,0.0,0.0,0.062500,0.0,0.015625,1258.875000,0.765625,1332.468750,0.0
2020-12-29,572.0,1712.0,2020,4307.664062,0.203125,0.015625,0.0,0.0,0.0,0.062500,0.0,0.015625,1228.265625,0.765625,1299.078125,0.0
2020-12-30,573.0,1719.0,2020,4224.437500,0.187500,0.015625,0.0,0.0,0.0,0.046875,0.0,0.015625,1199.109375,0.750000,1267.296875,0.0


In [174]:
# compare with GloFAS analysis

fig = go.Figure()
fig.update_layout(template="simple_white", title="Comparaison à Niamey")
fig.add_trace(
    go.Scatter(
        x=df_compare.index,
        y=df_compare["Discharges (m3/s)"],
        name="Observé (ABN)",
    )
)
fig.add_trace(
    go.Scatter(x=df_compare.index, y=df_compare["Niamey3"], name="GloFAS 3.1")
)
fig.add_trace(
    go.Scatter(x=df_compare.index, y=df_compare["Niamey4"], name="GloFAS 4.0")
)

ymax = df_compare[["Niamey3", "Niamey4", "Discharges (m3/s)"]].max().max()
fig.update_yaxes(title="Débit (m<sup>3</sup>/s)", range=[0, ymax * 1.1])


fig.show()

fig = go.Figure()
fig.update_layout(template="simple_white")
fig.add_trace(
    go.Scatter(
        x=df_compare["Water Level (cm)"],
        y=df_compare["Niamey4"],
        name="GloFAS",
        mode="markers",
    )
)
fig.show()

In [147]:
df_compare.corr()

,Water Level (cm),Discharges (m3/s),year,Niamey3,Alcongui,Boss Bangou,Dargol A Kakassi,Dargol At Tera,Diongore Amont,Garbe-Kourou,Gorouol At Dolbel,Goulbi De Maradi A Nielloua,Kandadji,Majia At Tsernaoua,Niamey4,W
Water Level (cm),1.000000,0.983697,NaN,0.415247,0.429186,0.538116,0.316302,0.291692,NaN,0.541301,0.413594,0.576765,0.433940,0.561183,0.484321,NaN
Discharges (m3/s),0.983697,1.000000,NaN,0.348511,0.507311,0.629437,0.357166,0.326460,NaN,0.633292,0.486486,0.657688,0.400725,0.644024,0.458894,NaN
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Niamey3,0.415247,0.348511,NaN,1.000000,-0.164398,-0.158059,-0.178701,-0.140360,NaN,-0.152776,-0.171324,0.159273,0.854524,0.103509,0.842082,NaN
Alcongui,0.429186,0.507311,NaN,-0.164398,1.000000,0.860742,0.840659,0.806145,NaN,0.860170,0.990952,0.586432,-0.009130,0.631043,0.083617,NaN
Boss Bangou,0.538116,0.629437,NaN,-0.158059,0.860742,1.000000,0.673815,0.609849,NaN,0.999249,0.837708,0.780647,0.135990,0.828396,0.222525,NaN
Dargol A Kakassi,0.316302,0.357166,NaN,-0.178701,0.840659,0.673815,1.000000,0.988349,NaN,0.667406,0.895163,0.474711,-0.068035,0.513652,0.009623,NaN
Dargol At Tera,0.291692,0.326460,NaN,-0.140360,0.806145,0.609849,0.988349,1.000000,NaN,0.605602,0.868396,0.431352,-0.046036,0.460958,0.026745,NaN
Diongore Amont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Garbe-Kourou,0.541301,0.633292,NaN,-0.152776,0.860170,0.999249,0.667406,0.605602,NaN,1.000000,0.835956,0.782856,0.142057,0.827491,0.228654,NaN
